## Install dependencies

First, let's set up the dependencies we need.

In [44]:
// show kernel version
"""
Kotlin Jupyter kernel version: ${notebook.kernelVersion}
Java Runtime Environment version: ${notebook.jreInfo.version}
"""


Kotlin Jupyter kernel version: 0.12.0.322
Java Runtime Environment version: 21


In [45]:
USE {
    repositories {
        mavenCentral()
    }
    dependencies {
        implementation("dev.langchain4j:langchain4j:1.0.0-beta2")
        implementation("dev.langchain4j:langchain4j-mcp:1.0.0-beta2")
        implementation("dev.langchain4j:langchain4j-vertex-ai-gemini:1.0.0-beta2")
        implementation("org.jetbrains:markdown-jvm:0.7.3")
    }
}
// list the library, if the library is not exist, restart kernel
notebook.currentClasspath.joinToString("\n")

/Users/gaplo917/Library/Caches/JetBrains/IntelliJIdea2024.3/kotlinNotebook/kotlin-notebook-standalone.eb20de96/kernels/0.12.0-322/kotlin-jupyter-script-classpath-shadowed-zip_extracted/kotlin-stdlib-1.9.23.jar
/Users/gaplo917/Library/Caches/JetBrains/IntelliJIdea2024.3/kotlinNotebook/kotlin-notebook-standalone.eb20de96/kernels/0.12.0-322/kotlin-jupyter-script-classpath-shadowed-zip_extracted/kotlin-reflect-1.9.23.jar
/Users/gaplo917/Library/Caches/JetBrains/IntelliJIdea2024.3/kotlinNotebook/kotlin-notebook-standalone.eb20de96/kernels/0.12.0-322/kotlin-jupyter-script-classpath-shadowed-zip_extracted/kotlinx-serialization-core-jvm-1.6.3.jar
/Users/gaplo917/Library/Caches/JetBrains/IntelliJIdea2024.3/kotlinNotebook/kotlin-notebook-standalone.eb20de96/kernels/0.12.0-322/kotlin-jupyter-script-classpath-shadowed-zip_extracted/annotations-13.0.jar
/Users/gaplo917/Library/Caches/JetBrains/IntelliJIdea2024.3/kotlinNotebook/kotlin-notebook-standalone.eb20de96/kernels/0.12.0-322/kotlin-jupyter-sc

## Setup Project ID and location

To use the Google Gemini API, we need to specify our Google Cloud project ID and the location (region) where we want to run the API. The project ID identifies your Google Cloud project, and the location determines where your API requests will be processed.

In [46]:
val projectId = "gaplotech" // Replace with your actual project ID
val location = "us-central1"

"""
projectId: $projectId
location: $location
"""


projectId: gaplotech
location: us-central1


## Choose model name and prompt

Google offers several Gemini models with different capabilities and performance characteristics. Here's a summary of the available models:
| Model                                | Inputs                          | Outputs                          | Use Case                                                                 |
|--------------------------------------|---------------------------------|----------------------------------|--------------------------------------------------------------------------|
| Gemini 2.0 Flash<br>gemini-2.0-flash | Text, Code, Images, Audio, Video, Video with Audio, PDF | Text, Audio (private preview), Images (private preview) | Workhorse model for all daily tasks. Strong overall performance and supports real-time streaming Live API. |
| Gemini 2.5 Pro Experimental<br>gemini-2.5-pro-exp-03-25 | Text, Images, Video, Audio, PDF | Text                             | Most advanced reasoning Gemini model, especially for multimodal understanding, coding, and world knowledge. |
| Gemini 2.0 Flash-Lite<br>gemini-2.0-flash-lite | Text, Images, Video, Audio, PDF | Text                             | Our cost effective offering to support high throughput.                  |
| Gemini 2.0 Flash Thinking<br>gemini-2.0-flash-thinking-exp-01-21 | Text, Images                    | Text                             | Provides stronger reasoning capabilities and includes the thinking process in responses. |

In [47]:
// Define the model to use
val modelName = "gemini-2.0-flash-exp"


## Import required classes for this notebook

In [48]:
import org.intellij.markdown.flavours.commonmark.CommonMarkFlavourDescriptor
import org.intellij.markdown.html.HtmlGenerator
import org.intellij.markdown.parser.MarkdownParser
import dev.langchain4j.mcp.McpToolProvider
import dev.langchain4j.mcp.client.DefaultMcpClient
import dev.langchain4j.mcp.client.McpClient
import dev.langchain4j.mcp.client.transport.http.HttpMcpTransport
import dev.langchain4j.memory.chat.MessageWindowChatMemory
import dev.langchain4j.model.vertexai.VertexAiGeminiChatModel
import dev.langchain4j.service.AiServices
import dev.langchain4j.service.SystemMessage
import dev.langchain4j.service.UserMessage
import dev.langchain4j.service.V

// a bridge to render Markdown text to HTML in the kernal
class Markdown(val content: String) : Renderable {
    companion object {
        private val flavour = CommonMarkFlavourDescriptor()
        private val mdParser = MarkdownParser(flavour)
    }

    override fun render(notebook: Notebook): DisplayResult {
        return HTML(HtmlGenerator(content, mdParser.buildMarkdownTreeFromString(content), flavour).generateHtml())
    }
}

## Create Langchain4j client with in-memory chat memory

The following implementation is to connect the Langchain4j MCP client to MCP Server via http://localhost:3001/sse.

In [72]:
interface Supervisor {
    @SystemMessage("""You are a helpful ai assistant with a list of tools.
     Always Ask follow up questions if it is ambiguous.
     Use avaliable tools automatically as parallel as possible.""")
    @UserMessage("{{message}}")
    fun chat(@V("message") userMessage: String): String
}

val transport = HttpMcpTransport.Builder()
    .sseUrl("http://localhost:3001/sse")
    .build()

val mcpClient: McpClient = DefaultMcpClient.Builder()
    .transport(transport)
    .build()

val toolProvider = McpToolProvider.builder()
    .mcpClients(listOf(mcpClient))
    .build();

val model = VertexAiGeminiChatModel.builder()
    .project(projectId)
    .location(location)
    .modelName(modelName)
    .build()

// The in-memory chat memory
val chatMemory = MessageWindowChatMemory.withMaxMessages(10)

val bot = AiServices.builder(Supervisor::class.java)
    .chatLanguageModel(model)
    .chatMemory(chatMemory)
    .toolProvider(toolProvider)
    .build()


## Create a chat function that always render in Markdown

In [73]:
fun chat(message: String) {
    DISPLAY(Markdown(bot.chat(message)))
}

## Start multi-turn conversation with memory

In [74]:
// always clear chat memory on the first turn
chatMemory.clear()

// 1st turn
chat("Get Bitcoin price today")

I found several cryptocurrencies related to Bitcoin. Which one are you looking for? I can search for: Bitcoin (BTC) Wrapped Bitcoin (WBTC) Bitcoin Cash (BCH) Bitcoin SV (BSV) and many more.

In [60]:
chat("BTC")

Bitcoin (BTC) is currently trading at $80,903 on Binance, with a volume of 60177.6983 BTC. The price may vary slightly depending on the exchange.

In [61]:
chat("Do you have other exchange price?")

Certainly! Here are some prices for Bitcoin (BTC) on other exchanges: Pionex: $81,125 WEEX: $80,975 CoinW: $81,030 OKX: $80,893 Crypto.com Exchange: $80,900 Bybit: $80,981 Coinbase Exchange: $80,962 Bitget: $80,809 DigiFinex: $80,845 Please note that these prices were last fetched within the last few minutes, but they may still vary slightly due to the constant fluctuations in the cryptocurrency market.

In [62]:
chat("Is there a gap I can make money base on the price difference?")

Cryptocurrency prices can vary across exchanges due to differences in supply, demand, trading fees, and other factors. While these price differences, also known as arbitrage opportunities, can potentially be profitable, several risks and challenges are associated with exploiting them: Transaction Fees: Every transaction on an exchange involves fees. If these fees are higher than the price difference, the arbitrage opportunity disappears. Withdrawal and Deposit Times: Moving funds between exchanges takes time. During this time, the price difference might disappear or even reverse, leading to a loss. Trading Speed: executing trades quickly is crucial. Delays can result in missing the opportunity or trading at a less favorable price. Exchange Limits and Verification: Exchanges often have limits on withdrawals and deposits, especially for new or unverified accounts. These limits can hinder your ability to capitalize on arbitrage opportunities. Slippage: Slippage refers to the difference between the expected price of a trade and the actual price at which the trade is executed. High slippage can erode potential profits. Market Volatility: Cryptocurrency markets are highly volatile. Prices can change rapidly, making arbitrage opportunities fleeting. Regulatory Issues: Regulations regarding cryptocurrency trading vary by jurisdiction and exchange. Ensure you comply with all applicable regulations. Based on the data I have, the price differences appear to be small. Given the factors I mentioned, it may be difficult to make a profit with these differences.

In [63]:
chat("summarize what we have discussed")

Okay, here's a summary of our conversation: Bitcoin Price: We started by discussing the current price of Bitcoin (BTC) on different exchanges. I initially provided the price on Binance ($80,903) and then used my tool to find the price on several other exchanges. Price Variation: We observed that the price of Bitcoin varies across different exchanges. Arbitrage Opportunity: You inquired about the potential to profit from these price differences through arbitrage. Risks and Challenges: I outlined the various risks and challenges associated with arbitrage, including transaction fees, withdrawal/deposit times, trading speed, exchange limits, slippage, market volatility, and regulatory issues. Profitability Assessment: I concluded that, given the small price differences and the associated risks, it might be difficult to make a profit from arbitrage in this specific situation.

In [64]:
chatMemory.clear()

In [65]:
chat("summarize what we have discussed")

I'm sorry, but as a new chatbot, I have no memory of past conversations. To help me summarize, could you please provide me with the content of our previous discussion?

In [ ]:
chatMemory.clear()